In [1]:
import pandas as pd
import numpy as np
import matplotlib as mth

In [55]:
dollar=pd.read_excel("./datas/dolarBRL2011.2021.xlsx")
barril=pd.read_excel("./datas/petroleo.2011.2021.xlsx")

dollar['V.Médio']=(dollar['Máxima']+dollar['Mínima'])/2
dollar['Data']=pd.to_datetime(dollar['Data'],format='%d/%m/%Y')
dollar.sort_values('Data',inplace=True)
dollar.set_index('Data',inplace=True)
dollar.rename_axis('Período',inplace=True)

barril['V.Médio']=(barril['Máxima']+barril['Mínima'])/2
barril['Data']=pd.to_datetime(barril['Data'],format='%d/%m/%Y')
barril.sort_values('Data',inplace=True)
barril.set_index('Data',inplace=True)
barril.rename_axis('Período',inplace=True)

dollar['VUS.Norm']=dollar['V.Médio']/dollar['V.Médio'].max()
barril['VBar.Norm']=barril['V.Médio']/barril['V.Médio'].max()

In [2]:
df_gas = pd.read_csv('C:/Users/ednae/Documents/Linguagens/data2/df_gas.csv')
df_gas['Data da Coleta']=pd.to_datetime(df_gas['Data da Coleta'],format="%d/%m/%Y")
df_gas.sort_values('Data da Coleta',inplace=True)
df_gas.set_index('Data da Coleta',drop=False,inplace=True)
df_gas.rename_axis('Período',inplace=True)
df_gas.drop(['Municipio','Complemento'],axis=1,inplace=True)
df_gas.drop(['Bandeira'],axis=1,inplace=True)
df_gas.drop(['Município','Cep','Número Rua','Numero Rua','Regiao - Sigla','Região - Sigla'],axis=1,inplace=True)

df_gas['ano']=df_gas['Data da Coleta'].dt.year
df_gas['mes']=df_gas['Data da Coleta'].dt.month
df_gas['dia']=df_gas['Data da Coleta'].dt.day

C:\Users\ednae\anaconda3\envs\rsd\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (1,3,7,17,18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## <b><i> Justificando uso da média, mostrando que o desvio padrão é baixo

In [4]:
mix_resumo=df_gas.groupby(['ano','mes','dia']).agg({'Valor de Venda':['mean','std','max','min']})
print(mix_resumo['Valor de Venda']['std'].max(),'desvio padrão máximo em todas as médias diárias calculadas')

0.4396596627479246 desvio padrão máximo em todas as médias diárias calculadas


In [6]:
df_gas2 = pd.DataFrame(df_gas['Valor de Venda'].resample('D').mean())
df_gas2['pct_change']=df_gas2.pct_change()*100
dif_med_mensal = df_gas2['pct_change'].resample('M').mean()

In [143]:
gas_med_mensal = df_gas['Valor de Venda'].resample('M').mean()

In [17]:
aaa=dif_med_mensal.to_frame()

In [22]:
type(dollar['pct_change'])

pandas.core.series.Series

In [56]:
barril['pct_change']=barril['V.Médio'].pct_change()*100
dollar['pct_change']=dollar['V.Médio'].pct_change()*100

In [180]:
barril.loc['2016':,'pct_change'].describe()

count    69.000000
mean      1.430261
std      10.088836
min     -31.349387
25%      -3.778461
50%       2.695999
75%       7.244795
max      29.545455
Name: pct_change, dtype: float64

# <b> Correlações dos valores brutos

In [145]:
comp=pd.DataFrame({'bar':barril.loc[:'2015','V.Médio'],'dol':dollar.loc[:'2015','V.Médio'],'gas':gas_med_mensal[:'2015']})
comp.reset_index(inplace=True)
comp['mes']=comp['Período'].dt.month
comp['ano']=comp['Período'].dt.year
resum=pd.DataFrame(comp.groupby(['mes','ano']).agg(['mean']))
resum.corr()#controle de preço

,,bar,dol,gas
,,mean,mean,mean
bar,mean,1.000000,-0.898977,-0.879464
dol,mean,-0.898977,1.000000,0.938624
gas,mean,-0.879464,0.938624,1.000000


In [149]:
comp=pd.DataFrame({'bar':barril.loc['2016':,'V.Médio'],'dol':dollar.loc['2016':,'V.Médio'],'gas':gas_med_mensal['2016':]})
comp.reset_index(inplace=True)
comp['mes']=comp['Período'].dt.month
comp['ano']=comp['Período'].dt.year
resum2=pd.DataFrame(comp.groupby(['mes','ano']).agg(['mean']))
resum2.corr()#PPI

,,bar,dol,gas
,,mean,mean,mean
bar,mean,1.000000,-0.131504,0.594711
dol,mean,-0.131504,1.000000,0.649542
gas,mean,0.594711,0.649542,1.000000


In [151]:
import scipy.stats as sst

In [ ]:
scipy.stats.pearsonr(comp['bar'],comp['gas'])

# <b> Correlações das variações mensais

In [142]:
comp=pd.DataFrame({'var_bar':barril.loc[:'2015','pct_change'],'var_dol':dollar.loc[:'2015','pct_change'],'var_gas':dif_med_mensal[:'2015']})
comp.reset_index(inplace=True)
comp['mes']=comp['Período'].dt.month
comp['ano']=comp['Período'].dt.year
resum3=pd.DataFrame(comp.groupby(['mes','ano']).agg(['mean']))
resum3.corr()#controle de preço

,,var_bar,var_dol,var_gas
,,mean,mean,mean
var_bar,mean,1.000000,-0.247628,0.131917
var_dol,mean,-0.247628,1.000000,-0.043486
var_gas,mean,0.131917,-0.043486,1.000000


In [141]:
comp=pd.DataFrame({'var_bar':barril.loc['2016':,'pct_change'],'var_dol':dollar.loc['2016':,'pct_change'],'var_gas':dif_med_mensal['2016':]})
comp.reset_index(inplace=True)
comp['mes']=comp['Período'].dt.month
comp['ano']=comp['Período'].dt.year
resum4=pd.DataFrame(comp.groupby(['mes','ano']).agg(['mean']))
resum4.corr()#PPI

,,var_bar,var_dol,var_gas
,,mean,mean,mean
var_bar,mean,1.000000,-0.50715,0.505854
var_dol,mean,-0.507150,1.00000,-0.116790
var_gas,mean,0.505854,-0.11679,1.000000
